In [5]:
import pandas as pd


  Could not find a version that satisfies the requirement BeautilfulSoup (from versions: )
No matching distribution found for BeautilfulSoup


  Could not find a version that satisfies the requirement time (from versions: )
No matching distribution found for time


In [1]:
import requests
from bs4 import BeautifulSoup
import bs4
import time
import numpy as np
import re
import time
#the goal is scrapping wikipedia in order to obtain information about citrus fruit
#We take the description section of each one and then we will extract main info.


In [53]:

#A function that gets the URL of the page to be scraped
#,gets the html content and uses BeautifulSoup to parse html content

def make_soup(link):
    get_page = requests.get(link)
    html = get_page.content
    soup = BeautifulSoup(html, 'html.parser')
    return  soup


#####This function create a list with all the link of the foods in a wikipedia Page.
def make_link_list(wiki_page_to_scrap):
    start_time = time.time()
    link_table = []
    soup = make_soup(wiki_page_to_scrap)
    table = soup.find('table',{'class':'wikitable'})
    
            ### This first loop is used to scrap Wiki table Data.
    if isinstance(table , bs4.element.Tag):        
    
        table_cells = []
        table = soup.find('table',{'class':'wikitable'})
        for row in table.find_all("tr"):
            cells = row.find_all(['th' , 'td'])
            table_cells.append(cells)
        
        ### This loop is used to locate the "Common name" column index in our table cells
        indices = []
        for j in table_cells:
            for i, elem in enumerate(j):
                elem = str(elem)
                if 'name' in elem:
                    indices.append(i)
        indice = indices[0]
        
        #print('Table Cells' , table_cells[:3])
        ### Here we implement a loop to keep only the string of the Common name column.
        
        for cell in table_cells[2:]:
            
            if (len(cell) < indice) == True :
                pass
             
            else:    
                link_table.append(cell[indice].text)
            
        ### Let's clean the table.
        for link in range(len(link_table)):
            link_table[link] = link_table[link].strip('\n')
            
        print('cpu time for the table schema = {:.4f} sec.'.format(time.time() - start_time))
            
            ### Here is when the Wiki page is just an Alphabetical List.    
    else:
        print(soup.find_all('div' , {'class':'div-col'}))
        for row in soup.find_all('div' , {'class':'div-col'}):
            
            for col in row.find_all('li'):
                species = col.text
                ###We just keep the common name of the species, because only the common name is used in recipes.
                only_common_species = re.sub("[\(\[].*?[\)\]]", "", species) 
                only_common_species , sep , tail = only_common_species.partition(',')
                link_table.append(only_common_species)
                
                ###Cleaning of the list, we remove all the occurence of string begining by List.
        for word in link_table[:]:
            if word.find('List') != -1:
                link_table.remove(word)
            
        print('cpu time for the list schema = {:.4f} sec.'.format(time.time() - start_time))     

    return link_table  



In [54]:
#page_citrus = 'https://en.wikipedia.org/wiki/List_of_citrus_fruits'
#list_citrus = make_link_list(page_citrus)
#number_citrus = len(list_citrus)
#print('Number of citrus' , len(list_citrus))

#page_salads = 'https://en.wikipedia.org/wiki/List_of_leaf_vegetables'
#list_salad = make_link_list(page_salads)
#number_salad = len(list_salad)
#print('Number of salads' , len(list_salad))

page_spices = 'https://en.wikipedia.org/wiki/List_of_culinary_herbs_and_spices'
#list_spices = make_link_list(page_spices)
#number_spices = len(list_spices)
#print('Number of spices' , len(list_spices))
#print(list_spices[:3])

#page_fruit = 'https://simple.wikipedia.org/wiki/List_of_fruits'
#list_fruit = make_link_list(page_fruit)
#number_fruit = len(list_fruit)
#print('Number of fruits', len(list_fruit))

#page_herbs = 'https://simple.wikipedia.org/wiki/List_of_herbs'
#list_herbs = make_link_list(page_herbs)
#number_herbs = len(list_herbs)
#print('Number of herbs', len(list_herbs))

page_root_vegetable = 'https://en.wikipedia.org/wiki/List_of_root_vegetables'
list_root_vegetable = make_link_list(page_root_vegetable)
number_root = len(list_root_vegetable)
print('Number of root vegetables' , len(list_root_vegetable))
print(list_root_vegetable[:4])


#print('Total number' , number_citrus + number_fruit + number_herbs + number_root + number_salad + number_spices )

[]
cpu time for the list schema = 10.0026 sec.
Number of root vegetables 0
[]


In [15]:
# Let's create a dictionary with the different categories of vegetables. 


foods_dict = dict([('CITRUS' , list_citrus) , ('SALAD' , list_salad) , ('SPICES' , list_spices)])



In [21]:
page_vegetable = 'https://simple.wikipedia.org/wiki/List_of_vegetables'
list_vegetable = make_link_list(page_vegetable)
print('Number of vegetables' , len(list_vegetable))

cpu time for the list schema = 0.1300 sec.
Number of vegetables 0


In [13]:
page_fruit = 'https://simple.wikipedia.org/wiki/List_of_fruits'

list_fruit = make_link_list(page_fruit)
print('List of fruit' , list_fruit[:4] , len(list_fruit))

cpu time for the list schema = 0.1417 sec.
List of fruit ['Açaí', 'Apple', 'Akee', 'Apricot'] 114
